In [18]:
import os
#IMPORT DATASET FROM KAGGLE
os.environ['KAGGLE_USERNAME'] = "XXX" # username from the json file
os.environ['KAGGLE_KEY'] = "XXXX" # key from the json file
#!kaggle competitions download -c titanic

In [35]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


In [36]:
#Criação colunas

X["Family"] = X["Name"]
#SibSp: esposa / Parch: pais/filhos
#Informações sobre parentes e se estava sozinho ou não
X["Alone"] = X['Alone'] = np.where(X["SibSp"] + X["Parch"] > 0, 0, 1)


In [37]:
#Criação colunas

test["Family"] = test["Name"]
#SibSp: esposa / Parch: pais/filhos
#Informações sobre parentes e se estava sozinho ou não
test["Alone"] = test['Alone'] = np.where(test["SibSp"] + test["Parch"] > 0, 0, 1)


In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

def extraiFamilia(nome):
    return nome.split(',')[0].strip()    

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True, excluirFamily=True, excluirAlone=True):
        self.excluirName = excluirName
        self.excluirFamily = excluirFamily
        self.excluirAlone = excluirAlone
    def fit(self, X, y=None):
        #Identificação da familia
        X["Family"] = X["Name"]
        X['Alone'] = 0
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        
        if self.excluirFamily:
            self.colunasIndesejadas.append('Family')
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        if self.excluirAlone:
            self.colunasIndesejadas.append('Alone')    
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        if 'Family' not in self.colunasIndesejadas:
            Xdrop['Family'] = Xdrop['Family'].apply(extraiFamilia)
        if 'Alone' not in self.colunasIndesejadas:
            Xdrop['Alone'] = np.where(Xdrop["SibSp"] + Xdrop["Parch"] > 0, 0, 1)
            Xdrop = Xdrop.drop('SibSp',axis=1)
            Xdrop = Xdrop.drop('Parch',axis=1)

        return Xdrop


In [39]:
#Teste
atributosDesejados = AtributosDesejados(excluirName=False,excluirFamily=False,excluirAlone=False)
a = atributosDesejados.fit_transform(X)
a.head()

,Pclass,Name,Sex,Age,Fare,Embarked,Family,Alone
0,3,Mr,male,22.0,7.2500,S,Braund,0
1,1,Mrs,female,38.0,71.2833,C,Cumings,0
2,3,Miss,female,26.0,7.9250,S,Heikkinen,1
3,1,Mrs,female,35.0,53.1000,S,Futrelle,0
4,3,Mr,male,35.0,8.0500,S,Allen,1


In [40]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [41]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [42]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier(n_jobs=-1))
])

parametros = {
    'atributosDesejados__excluirName': [True, False],
    'atributosDesejados__excluirFamily': [True, False],
    'atributosDesejados__excluirAlone': [True, False],
    'classificador__max_depth': [15],
    #'classificador__criterion': ['gini', 'entropy'],
    'classificador__n_estimators': [100]
}
modelo = GridSearchCV(pipetotal, param_grid=parametros)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold(),scoring="roc_auc")
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

A saída de streaming foi truncada nas últimas 5000 linhas.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(array([0.84967949, 0.87156085, 0.85742794, 0.89794974, 0.85993187,
        0.86021722, 0.90006684, 0.8419244 , 0.87671327, 0.83402346,
        0.87496626, 0.8715443 , 0.86997958, 0.8635582 , 0.87719537,
        0.85280374, 0.87514124, 0.9002193 , 0.86660144, 0.85029354,
        0.89025208, 0.86586811, 0.86394605, 0.86453645, 0.86478626,
        0.8761085 , 0.83662281, 0.86775468, 0.87619048, 0.87687807,
        0.8891097 , 0.87412067, 0.87703092, 0.79959156, 0.90338291,
        0.87136304, 0.83121693, 0.85146358, 0.88424638, 0.92802589,
        0.88705092, 0.89155107, 0.85948276, 0.88699487, 0.83007288,
        0.86480051, 0.92557566, 0.83660901, 0.86303804, 0.85883424]),
 0.8689660615900483,
 0.023291130524214124)

In [ ]:
#test["Family"] = test["Name"].map(lambda x: x.split(',')[0].strip())

In [44]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [46]:
!kaggle competitions submit -c titanic -f submission.csv -m "new submission"

100% 2.77k/2.77k [00:01<00:00, 1.70kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

Foram feitas diversas tentativas em outro notebook, no qual adicionei novas colunas calculadas, sendo elas, família, quantidade de parentes, se pessoa sozinha e o deck da cabine da pessoa. 

O notebook está disponível em: https://github.com/rodseidel/mpca/blob/master/ReconhecimentoPadroes/REPA_w7_titanic_revis%C3%A3o_generaliza%C3%A7%C3%A3o1.ipynb

Os resultados foram:

Adicionado colunas alone e relativies:
- média test-score: 0.8306295901073381
- Kaggle submission: 0.78468


Adicionando coluna Deck:
- média test-score: 0.8280679178959263
- Kaggle submission: 0.78708


Porém, as opções que adicionei ao GridSearchCV tornaram o processo muito lento para novas tentativas. 


Diante disso preparei este notebook e fazendo a criação da coluna Família, bem como a inclusão de novos parâmetros no GridSearchCV, bem como parâmetros de busca no modelo, cheguei ao resultado:

- média test-score: 0.8698945617526356
- Kaggle submission: 0.79425